# The Cahart Four Factor Model #

### Using Relative Strenght to approach Momentum ###

In [4]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_stock_universe
from capm_toolkit import compute_daily_returns
from capm_toolkit import compute_excess_returns
from capm_toolkit import capm_regression
from portfolios_toolkit import calculate_analytics

In [8]:
# Get the important data for the Risk Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)

# Get the important data for the S&P500
sp500 = pd.read_csv(r"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

# Get the data for the Stocks' Betas
betas_df = pd.read_csv(r"..\additional_data\capm_hbetas.csv")
betas_df = betas_df.set_index('date')
betas_df.index = pd.to_datetime(betas_df.index)

In [3]:
# Dictionary to store the DataFrames
folder_path = r"..\stocks"

dataframes = import_stock_universe(
    folder_path,
    ['Adjusted_close', 'Company Market Cap'],
    ['adj_close', 'mkt_cap'],
)

In [9]:
# Create a whole new dataframe that contains all the stocks betas
rets_series = []

for stock, df in dataframes.items():
    series = df['adj_close'].pct_change(1).rename(stock)  
    series = series.iloc[1:]
    rets_series.append(series)

# Concat
returns_df = pd.concat(rets_series, axis=1)
returns_df = returns_df.apply(lambda x: x.fillna(x.mean()), axis=0)

# Drop nans
returns_df.dropna(inplace = True)

returns_df

In [10]:
# Create a whole new dataframe that contains all the stocks betas
mktcap_series = []

for stock, df in dataframes.items():
    series = df['mkt_cap'].rename(stock)  
    mktcap_series.append(series)

# Concat
mktcap_df = pd.concat(mktcap_series, axis=1)

# Apply Logs and EMA (maybe)
mktcap_df = np.log(mktcap_df)
mktcap_df = mktcap_df.ewm(span=5, adjust = False).mean()
mktcap_df = mktcap_df.bfill()
mktcap_df

In [11]:
# Create the Weights function
def wexp(N, half_life):
    c = np.log(0.5)/half_life
    n = np.array(range(N))
    w = np.exp(c*n)
    return np.flip(w/np.sum(w))

def n_days_nonmiss(returns, tiny_ret=1e-6):
    ix_ret_tiny = np.abs(returns) <= tiny_ret
    return np.sum(~ix_ret_tiny, axis=0)

def calc_rstr(returns, half_life=126, min_obs=100, tiny_ret=1e-6):
    rstr = np.log(1.+returns)
    if half_life == 0:
        weights = np.ones_like(rstr)
    else:
        weights = len(returns) * np.asmatrix(wexp(len(returns),half_life)).T
    rstr = np.sum(rstr * weights)
    idx = n_days_nonmiss(returns) < min_obs
    rstr.where(~idx, other=np.nan, inplace=True)
    df = pd.Series(rstr)
    df.name = returns.index[-1]
    return df

def rolling_calc_rstr(
        returns,
        window_size=252,
        half_life=126,
        min_obs=100
):
    rolling_results = []
    range_to_iter = range(len(returns) - window_size + 1)
    for i in range_to_iter:
        window_returns = returns.iloc[i:i + window_size]
        rs_i = calc_rstr(
            returns=window_returns,
            half_life=half_life,
            min_obs=min_obs
        )

        rolling_results.append(rs_i)

    return pd.concat(rolling_results, axis=1)


In [12]:
# Create a whole new dataframe that contains all the stocks betas
relative_strenght_long = rolling_calc_rstr(
    returns_df,
    window_size=252,
    half_life=126
).T

relative_strenght_short = rolling_calc_rstr(
    returns_df,
    window_size=28,
    half_life=14,
    min_obs=13
).T

In [13]:
relative_strenght = (relative_strenght_long - relative_strenght_short).dropna()

relative_strenght

In [15]:
# Create Plot

tickers = ['NVDA', 'MSFT', 'GOOG', 'AAPL', 'AMZN']

plt.figure(figsize=(10, 6))
plt.plot(relative_strenght[tickers], label=tickers, alpha=1)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Relative Strength Time Series')
plt.xlabel('Time')
plt.ylabel('Relative Strength')
plt.legend()
plt.grid()

# Show
plt.show()

In [125]:
# Store the Data

# relative_strenght.to_csv(r"..\additional_data\momentum.csv")

### Calculate the Momentum Portfolios ###

In [16]:
# Define the Decomposition Function
def momentum_decomposition(
    target_df, 
    mktcap_df, 
    momentum_df
):
    # Common Indexes
    common_index = target_df.index.intersection(momentum_df.index).intersection(mktcap_df.index)
    
    # Reindex
    target_df = target_df.loc[common_index]
    mktcap_df = mktcap_df.loc[common_index]
    momentum_df = momentum_df.loc[common_index]

    # Initialize lists to store portfolio returns
    winner_list, neutral_list, loser_list = [], [], []
    
    # Get unique quarters
    months = sorted(set([date.to_period('M') for date in common_index]))
    
    # Dictionary to store quarterly classifications and weights
    monthly_classifications = {}

    for month in months:
        # Select only the last available date of the quarter
        month_dates = [date for date in common_index if date.to_period('M') == month]
        rebalance_date = month_dates[-1]  # Last day of the quarter
        
        # Momentum Factor for rebalance date
        momentum_factor_df = pd.DataFrame([momentum_df.loc[rebalance_date]], index=['mom']).T.dropna()
        
        # Classify stocks into Low, Neutral, and High based on quantiles
        lower = momentum_factor_df['mom'].quantile(0.3)
        upper = momentum_factor_df['mom'].quantile(0.7)

        momentum_factor_df['momentum_class'] = 'neutral'
        momentum_factor_df.loc[momentum_factor_df['mom'] <= lower, 'momentum_class'] = 'loser'
        momentum_factor_df.loc[momentum_factor_df['mom'] >= upper, 'momentum_class'] = 'winner'
        
        # Market cap data
        market_caps_df = pd.DataFrame([mktcap_df.loc[rebalance_date]], index=['mkt_cap']).T
        
        # Assign market caps to value classes
        loser_mktcap_df = market_caps_df.loc[momentum_factor_df[momentum_factor_df['momentum_class'] == 'loser'].index]
        neutral_mktcap_df = market_caps_df.loc[momentum_factor_df[momentum_factor_df['momentum_class'] == 'neutral'].index]
        winner_mktcap_df = market_caps_df.loc[momentum_factor_df[momentum_factor_df['momentum_class'] == 'winner'].index]
        
        # Compute weights
        loser_weights = loser_mktcap_df['mkt_cap'] / loser_mktcap_df['mkt_cap'].sum()
        neutral_weights = neutral_mktcap_df['mkt_cap'] / neutral_mktcap_df['mkt_cap'].sum()
        winner_weights = winner_mktcap_df['mkt_cap'] / winner_mktcap_df['mkt_cap'].sum()
        
        # Store classifications and weights
        monthly_classifications[month] = {
            "loser": loser_weights,
            "neutral": neutral_weights,
            "winner": winner_weights
        }
    
    # Iterate over all available dates to compute daily returns
    for date in common_index:
        month_key = date.to_period('M')  # Get quarter of the current date
        
        if month_key in monthly_classifications:
            # Retrieve stored classification and weights
            loser_weights = monthly_classifications[month_key]["loser"]
            neutral_weights = monthly_classifications[month_key]["neutral"]
            winner_weights = monthly_classifications[month_key]["winner"]
            
            # Retrieve daily returns
            target = pd.DataFrame([target_df.loc[date]], index=['returns']).T
            
            loser_returns = target.reindex(loser_weights.index).dropna()
            neutral_returns = target.reindex(neutral_weights.index).dropna()
            winner_returns = target.reindex(winner_weights.index).dropna()
            
            # Compute portfolio returns
            loser_result = loser_weights.reindex(loser_returns.index).T @ loser_returns
            neutral_result = neutral_weights.reindex(neutral_returns.index).T @ neutral_returns
            winner_result = winner_weights.reindex(winner_returns.index).T @ winner_returns
            
            # Store results
            loser_list.append(loser_result.values[0] if not loser_result.empty else None)
            neutral_list.append(neutral_result.values[0] if not neutral_result.empty else None)
            winner_list.append(winner_result.values[0] if not winner_result.empty else None)

    # Create final DataFrame
    momentum_portfolios = pd.DataFrame({
        'winner': winner_list,
        'neutral': neutral_list,
        'loser': loser_list
    }, index=common_index)
    
    return momentum_portfolios

In [17]:
# Create DataFrames

momentum_portfolios_returns = momentum_decomposition(returns_df, mktcap_df, relative_strenght)

momentum_portfolios_returns

In [18]:
# Calculate the Analytics
mom_portfolios_analytics = calculate_analytics(momentum_portfolios_returns)

mom_portfolios_analytics

In [19]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(momentum_portfolios_returns.cumsum(), label=momentum_portfolios_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns (Momentum Adjusted) Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [20]:
# Create DataFrames

momentum_portfolios_betas = momentum_decomposition(betas_df, mktcap_df, relative_strenght)

momentum_portfolios_betas

In [21]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(momentum_portfolios_betas.ewm(span=21, adjust = False).mean(), label=momentum_portfolios_betas.columns, alpha=1)
plt.axhline(y=1, color='black', linestyle='dashed')

# Config
plt.title('Betas (Size Adjusted) Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()
plt.grid()

# Show
plt.show()

In [22]:
# Create the data
daily_rfr = compute_daily_returns(rfr['risk_free_rate'])
market_excess_returns = compute_excess_returns(sp500['sp_500'], rfr['risk_free_rate'])

In [23]:
# Create the regression dataframe
momentum_regression_df = pd.DataFrame(index = momentum_portfolios_returns.index)

momentum_regression_df['winners_excess_returns'] = momentum_portfolios_returns['winner'] - daily_rfr
momentum_regression_df['losers_excess_returns'] = momentum_portfolios_returns['loser'] - daily_rfr
momentum_regression_df['market_excess_returns'] = market_excess_returns
momentum_regression_df.dropna(inplace = True)

momentum_regression_df

In [24]:
# Calculate the Beta for the Winner Portfolio

y = momentum_regression_df['winners_excess_returns']
x = momentum_regression_df['market_excess_returns']

# Calculate Weights
window = len(y)

#Model specification
results = capm_regression(
    y,
    x,
    window,
    True
)
    
#here we check the summary
print(results.summary())

In [25]:
# Calculate the Beta for the Loser Portfolio

y = momentum_regression_df['losers_excess_returns']
x = momentum_regression_df['market_excess_returns']

# Calculate Weights
window = len(y)

#Model specification
results = capm_regression(
    y,
    x,
    window,
    True
)
    
#here we check the summary
print(results.summary())

In [26]:
# Calculate the WML Premium
momentum_regression_df['WML'] = momentum_portfolios_returns['winner'] - momentum_portfolios_returns['loser']

In [27]:
plt.figure(figsize=(10, 6))
plt.plot(momentum_regression_df['WML'].cumsum(), label='WML Premium', color = 'salmon', alpha=1)

# Config
plt.title('WML Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [28]:
# Check the Correlation with the Market

momentum_regression_df['WML'].corr(momentum_regression_df['market_excess_returns'])

In [29]:
# Calculate the Beta for the WML premium

y = momentum_regression_df['WML']
x = momentum_regression_df['market_excess_returns']

# Calculate Weights
window = len(y)

#Model specification
results = capm_regression(
    y,
    x,
    window,
    True
)
    
#here we check the summary
print(results.summary())

In [32]:
# Store the Premium
momentum_regression_df.index.name = "Date"
momentum_regression_df['WML'].to_csv(r"..\additional_data\carhart_wml.csv")